# Set the Enviroment

In [1]:
import json
import os

# Rutas a los archivos de secrets
file_path_openai = r"C:\Users\Sebastian Perez Vega\AppData\Roaming\Microsoft\UserSecrets\9f4d8329-11d3-4e0a-a512-efa17c5996fa\secrets.json"
file_path_serper = r"C:\Users\Sebastian Perez Vega\AppData\Roaming\Microsoft\UserSecrets\e0d07652-75ce-4d80-ba9a-fb98f5ba7375\secrets.json"

# Cargar OpenAI key
with open(file_path_openai, encoding='utf-8-sig') as f:
    openai_data = json.load(f)
openai_api_key = openai_data.get("ai_agents_openai")

# Cargar Serper key
with open(file_path_serper, encoding='utf-8-sig') as f:
    serper_data = json.load(f)
serper_api_key = serper_data.get("serper_api")

# Establecer como variables de entorno
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key

# Confirmar
#print("🔑 OpenAI Key cargada:", openai_api_key[:5], "...")
#print("🔑 Serper Key cargada:", serper_api_key[:5], "...")


In [6]:
# import libraries
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [7]:
# define the LLM
llm = ChatOpenAI(model_name = 'gpt-4o', temperature = 0.5)

# User inputs

In [8]:
# call for inputs: Interviewer, company, job position, job descriptions:
interviewer = input('Enter the name of the interviewer (if unknown include None): ')
company = input('Enter the name of the company: ')
job_position = input('Enter the job position: ')
job_description = input('Enter the job description: ')

Enter the name of the interviewer (if unknown include None):  Matthias Glöel
Enter the name of the company:  Beat81
Enter the job position:  Data Analytics Working Student
Enter the job description:  About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathematics, computer scie

# Research Crew

## Research AI Agent

In [10]:
# create research agent
research_agent = Agent(
    role = 'Research Agent',
    goal = 'conduct in-depth research',
    backstory = """As a Research Speciallist, your mission is to uncover detailed and relevant information.
    The ultimate goal is to meet the expectations of the task""",
    llm = llm,
    tools= [ScrapeWebsiteTool(), SerperDevTool()]
)

## Research Task

In [11]:
# define the task about researching the company
research_company_task = Task(
    description = f"""
    Research about {company}, their position in the sector and recent industry trends. Agent=Focus on recent news and developments from last 12 month""",
    expected_output = f"""A comprehensive report in Markdown with bulled point about {company} and it's industry""",
    agent = research_agent
)

In [12]:
# Research Interviewer Task
research_person_task = Task(
    description = f'Research about {interviewer} with a focus on their professional career',
    expected_output = f"""A comprehensive report in Markdown with bullet point about {interviewer}""",
    agent = research_agent
)

## AI Coach + Interview Prep

In [13]:
# Create a AI Interview Coach
coach_agent = Agent(
    role = 'Interview Coach',
    goal = f'I help the user prepare for the job interview for job {job_position} by grading the relevance of the answer',
    backstory = f'You are an expert in job interviews in companies like {company}',
    llm = llm
)

In [14]:
# Prepare the list of questions
define_questions_task = Task(
    description = f"""
    Define a list of questions based on the company {company}, job description {job_description} and interviewer {interviewer}""",
    expected_output = f"""A list of 20 questions in Markdown format that cover: Culture and team fit, Job position Fit, Background and ways of working, Growth Mindset""",
    agent = coach_agent,
    context = [research_company_task, research_person_task]
)

## Assemble the crew

In [15]:
crew = Crew(
    agents = [research_agent, coach_agent],
    tasks = [research_company_task, research_person_task, define_questions_task],
    verbose = True,
    process = Process.sequential
)

In [16]:
# kickoff
result = crew.kickoff({'topic': 'Write a list of question to prepare for the interview'})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ed0ace19-e2d0-47e9-894b-10a75efd02d1                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Agent                                                                                          │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Research about Beat81, their position in the sector and recent industry trends. Agent=Focus on recent      │
│  news and developments from last 12 month                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Agent                                                                                          │
│                                                                                                                 │
│  Thought: To provide a comprehensive report on Beat81, their position in the sector, and recent industry        │
│  trends, I will start by searching for recent news and developments about Beat81 from the last 12 months. This  │
│  will help gather relevant and up-to-date information.                                                          │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"Beat81 recent news 2023\"}"                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interview Coach                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Define a list of questions based on the company Beat81, job description About the Role As a Working        │
│  Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to  │
│  support our daily operations with analytical insights and help us to grow. You will be joining a dynamic,      │
│  fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate    │
│  with stakeholders to understand data requirements  Support teams with data and analytics requests and provide  │
│  analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in       │
│  marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your    │
│  Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g.             │
│  mathematics, computer science, data science, business informatics etc.)  Good knowledge of Google Sheets       │
│  Good Knowledge of SQL  Good Knowledge of Python  Knowledge of common data visualization tools (e.g. Tableau,   │
│  Lookerstudio etc.)  Knowledge of dbt, GCP, BigQuery is a plus  Fluent in English  OUR OFFER  WALK THE TALK:    │
│  Unlimited BEAT81 workouts for you and one family member/friend: we guarantee that you will get fit working     │
│  here :)  YOUR DEVICE: Everyone has their own preferences when it comes to technology - whether it’s Mac or     │
│  Windows - you choose your laptop. Additionally, we provide Bose noise-canceling headphones  HYBRID WORK: We    │
│  have a beautiful office in the heart of Berlin (Weinmeisterstraße) with free drinks, healthy snacks and        │
│  fruits. We like to spend time there together, but you are also welcome to work from home  FLEXIBLE WORKING     │
│  HOURS: Early bird or late riser? Shape your workday to align with your personal commitments and preferences    │
│  in coordination with your team and the business needs  TEAM RITUALS: Shared experiences build a strong team    │
│  culture. That’s why we have our usuals such as regular all hands meetings, Aperol Thursdays and joint          │
│  workouts - there is something for everyone  TEAM EVENTS: Our summer and Christmas parties have become as       │
│  legendary as our annual off-sites and the perfect occasion to spend quality time with team members and         │
│  interviewer Matthias Glöel                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interview Coach                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Culture and Team Fit                                                                                         │
│  1. Can you describe the team culture at Beat81 and how it supports collaboration and innovation?               │
│  2. How does Beat81 integrate community engagement into its company culture and daily operations?               │
│  3. What are some team rituals or events that you find most impactful for team bonding at Beat81?               │
│  4. How does Beat81 ensure a welcoming and inclusive environment for all employees and participants?            │
│  5. Can you share an example of how the team at Beat81 has adapted to a fast-paced and dynamic work             │
│  environment?                                                                                                   │
│                                                                                                                 │
│  # Job Position Fit                                                                                             │
│  6. What are the key challenges and expectations for a Working Student in the Analytics Team at Beat81?         │
│  7. How does this role contribute to the overall success and growth of Beat81?                                  │
│  8. Can you describe a typical day for a Working Student in the Analytics Team?                                 │
│  9. What tools and technologies does Beat81 primarily use for data analytics and visualization?                 │
│  10. How does the Analytics Team collaborate with other departments like Tech, Product, Marketing, and          │
│  Operations?                                                                                                    │
│                                                                                                                 │
│  # Background and Ways of Working                                                                               │
│  11. What specific skills or experiences from my background would be most beneficial for this role?             │
│  12. How does Beat81 support the professional development of its working students?                              │
│  13. Can you provide examples of past projects that were successfully completed by working students in the      │
│  Analytics Team?                                                                                                │
│  14. How does Beat81 approach the integration of new technologies and methodologies in data analytics?          │
│  15. What is the process for setting goals and measuring success for a working student in this role?            │
│                                                                                                                 │
│  # Growth Mindset                                                                                               │
│  16. How does Beat81 encourage continuous learning and growth among its employees?                              │
│  17. What opportunities are available for working students to take on leadership roles or larger projects?      │
│  18. How does Beat81 stay ahead of industry trends and incorporate them into its business model and offerings?  │
│  19. Can you share a time when the Analytics Team at Be

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a6b459be-02fa-4fc5-9134-618704cc2b7c                                                                     │
│  Agent: Interview Coach                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ed0ace19-e2d0-47e9-894b-10a75efd02d1                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Culture and Team Fit                                                                                         │
│  1. Can you describe the team culture at Beat81 and how it supports collaboration and innovation?               │
│  2. How does Beat81 integrate community engagement into its company culture and daily operations?               │
│  3. What are some team rituals or events that you find most impactful for team bonding at Beat81?               │
│  4. How does Beat81 ensure a welcoming and inclusive environment for all employees and participants?            │
│  5. Can you share an example of how the team at Beat81 has adapted to a fast-paced and dynamic work             │
│  environment?                                                                                                   │
│                                                                                                                 │
│  # Job Position Fit                                                                                             │
│  6. What are the key challenges and expectations for a Working Student in the Analytics Team at Beat81?         │
│  7. How does this role contribute to the overall success and growth of Beat81?                                  │
│  8. Can you describe a typical day for a Working Student in the Analytics Team?                                 │
│  9. What tools and technologies does Beat81 primarily use for data analytics and visualization?                 │
│  10. How does the Analytics Team collaborate with other departments like Tech, Product, Marketing, and          │
│  Operations?                                                                                                    │
│                                                                                                                 │
│  # Background and Ways of Working                                                                               │
│  11. What specific skills or experiences from my background would be most beneficial for this role?             │
│  12. How does Beat81 support the professional development of its working students?                              │
│  13. Can you provide examples of past projects that were successfully completed by working students in the      │
│  Analytics Team?                                                                                                │
│  14. How does Beat81 approach the integration of new technologies and methodologies in data analytics?          │
│  15. What is the process for setting goals and measuring success for a working student in this role?            │
│                                                                                                                 │
│  # Growth Mindset                                                                                               │
│  16. How does Beat81 encourage continuous learning and growth among its employees?                              │
│  17. What opportunities are available for working students to take on leadership roles or larger projects?      │
│  18. How does Beat81 stay ahead of industry trends and

## Exporting

In [17]:
# Check the crew outcome
display(Markdown(result.tasks_output[2].raw))

```markdown
# Culture and Team Fit
1. Can you describe the team culture at Beat81 and how it supports collaboration and innovation?
2. How does Beat81 integrate community engagement into its company culture and daily operations?
3. What are some team rituals or events that you find most impactful for team bonding at Beat81?
4. How does Beat81 ensure a welcoming and inclusive environment for all employees and participants?
5. Can you share an example of how the team at Beat81 has adapted to a fast-paced and dynamic work environment?

# Job Position Fit
6. What are the key challenges and expectations for a Working Student in the Analytics Team at Beat81?
7. How does this role contribute to the overall success and growth of Beat81?
8. Can you describe a typical day for a Working Student in the Analytics Team?
9. What tools and technologies does Beat81 primarily use for data analytics and visualization?
10. How does the Analytics Team collaborate with other departments like Tech, Product, Marketing, and Operations?

# Background and Ways of Working
11. What specific skills or experiences from my background would be most beneficial for this role?
12. How does Beat81 support the professional development of its working students?
13. Can you provide examples of past projects that were successfully completed by working students in the Analytics Team?
14. How does Beat81 approach the integration of new technologies and methodologies in data analytics?
15. What is the process for setting goals and measuring success for a working student in this role?

# Growth Mindset
16. How does Beat81 encourage continuous learning and growth among its employees?
17. What opportunities are available for working students to take on leadership roles or larger projects?
18. How does Beat81 stay ahead of industry trends and incorporate them into its business model and offerings?
19. Can you share a time when the Analytics Team at Beat81 overcame a significant challenge?
20. What qualities do you believe are essential for success in this role and within the company?

```

This list of questions is designed to help candidates assess their fit for the role at Beat81, understand the company culture, and explore opportunities for growth and development within the organization.

In [19]:
# Export each of the tasks output
# exporting the company report
file_name = f'{company}_report.txt'

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[0].raw)

# Export the interviewer report
file_name = f'{interviewer}_report.txt'

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[1].raw)

# Export the interview questions
file_name = f'{job_position}_questions.txt'

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[2].raw)


# Interview Prep

In [20]:
# import the questions from the directory
with open(f'{job_position}_questions.txt', 'r') as file:
    questions = file.read()

In [21]:
# see the questions
questions

'```markdown\n# Culture and Team Fit\n1. Can you describe the team culture at Beat81 and how it supports collaboration and innovation?\n2. How does Beat81 integrate community engagement into its company culture and daily operations?\n3. What are some team rituals or events that you find most impactful for team bonding at Beat81?\n4. How does Beat81 ensure a welcoming and inclusive environment for all employees and participants?\n5. Can you share an example of how the team at Beat81 has adapted to a fast-paced and dynamic work environment?\n\n# Job Position Fit\n6. What are the key challenges and expectations for a Working Student in the Analytics Team at Beat81?\n7. How does this role contribute to the overall success and growth of Beat81?\n8. Can you describe a typical day for a Working Student in the Analytics Team?\n9. What tools and technologies does Beat81 primarily use for data analytics and visualization?\n10. How does the Analytics Team collaborate with other departments like T

In [22]:
# AI agent that ask the question
interviewer_agent = Agent(
    role = 'Interviewer Agent',
    goal = f'Ask a question from {questions} to the user',
    backstory = f'You are an expert in job interviews in companies like {company}',
    llm = llm
)

In [23]:
# Interview task
interview_prep_task = Task(
    description = f'Ask a question from {questions} to the user',
    expected_output = f'Ask only one question at a time',
    agent = interviewer_agent,
    human_input = True
)

In [24]:
# Feedback Task
feedback_task = Task(
    description = """Give feedback to the user on it's answer""",
    expected_output = 'Markdown answer of what was good, could be improved and how o take it to the next level',
    agent = coach_agent,
    context = [interview_prep_task],
    human_input = True
)

## Interview Prep Crew

In [25]:
# Assemble the interview prep crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.sequential
)

In [27]:
result = crew.kickoff({'topic': f'Ask me all questions from {questions}'})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6ea598fd-300e-4eb4-97b7-9bcca5fd8d97                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interviewer Agent                                                                                       │
│                                                                                                                 │
│  Task: Ask a question from ```markdown                                                                          │
│  # Culture and Team Fit                                                                                         │
│  1. Can you describe the team culture at Beat81 and how it supports collaboration and innovation?               │
│  2. How does Beat81 integrate community engagement into its company culture and daily operations?               │
│  3. What are some team rituals or events that you find most impactful for team bonding at Beat81?               │
│  4. How does Beat81 ensure a welcoming and inclusive environment for all employees and participants?            │
│  5. Can you share an example of how the team at Beat81 has adapted to a fast-paced and dynamic work             │
│  environment?                                                                                                   │
│                                                                                                                 │
│  # Job Position Fit                                                                                             │
│  6. What are the key challenges and expectations for a Working Student in the Analytics Team at Beat81?         │
│  7. How does this role contribute to the overall success and growth of Beat81?                                  │
│  8. Can you describe a typical day for a Working Student in the Analytics Team?                                 │
│  9. What tools and technologies does Beat81 primarily use for data analytics and visualization?                 │
│  10. How does the Analytics Team collaborate with other departments like Tech, Product, Marketing, and          │
│  Operations?                                                                                                    │
│                                                                                                                 │
│  # Background and Ways of Working                                                                               │
│  11. What specific skills or experiences from my background would be most beneficial for this role?             │
│  12. How does Beat81 support the professional development of its working students?                              │
│  13. Can you provide examples of past projects that were successfully completed by working students in the      │
│  Analytics Team?                                                                                                │
│  14. How does Beat81 approach the integration of new technologies and methodologies in data analytics?          │
│  15. What is the process for setting goals and measuring success for a working student in this role?            │
│                                                                                                                 │
│  # Growth Mindset                                                                                               │
│  16. How does Beat81 encourage continuous learning and growth among its employees?                              │
│  17. What opportunities are available for working students to take on leadership roles or larger projects?      │
│  18. How does Beat81 stay ahead of industry trends and incorporate them into its business model and offerings?  │
│  19. Can you share a time when the Analytics Team at Beat81 overcame a significant challenge?                   │
│  20. What qualities do you believe are essential for su

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interviewer Agent                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Can you describe the team culture at Beat81 and how it supports collaboration and innovation?                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 ## Final Result: Can you describe the team culture at Beat81 and how it supports collaboration and innovation?
 

=====
## HUMAN FEEDBACK: Provide feedback on the Final Result and Agent's actions.
Please follow these guidelines:
 - If you are happy with the result, simply hit Enter without typing anything.
 - Otherwise, provide specific improvement requests.
 - You can provide multiple rounds of feedback until satisfied.
=====



...

 
Processing your feedback...


Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interviewer Agent                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  How does Beat81 ensure a welcoming and inclusive environment for all employees and participants?               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 ## Final Result: How does Beat81 ensure a welcoming and inclusive environment for all employees and participants?
 

=====
## HUMAN FEEDBACK: Provide feedback on the Final Result and Agent's actions.
Please follow these guidelines:
 - If you are happy with the result, simply hit Enter without typing anything.
 - Otherwise, provide specific improvement requests.
 - You can provide multiple rounds of feedback until satisfied.
=====



KeyboardInterrupt: Interrupted by user

In [ ]:
# This is so you can use the 20 questions
# Assemble the interview prep crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.hierarchical,
    manager_llm = ChatOpenAI(model_name = 'gpt-4o', temperature = 0),
    memory = True,
    planning = True
)

In [ ]:
result = crew.kockoff({'topic': f'Ask me all questions from {questions}'})